# Oktoberfest Rescoring API Tutorial

This notebook covers how to run Oktoberfest Rescoring by using API functions. All API functions as well as additional guides and examples can be found on the official read the docs page at https://oktoberfest.readthedocs.io/en/stable/index.html.

The workflow is divided into six sections:

1. Install Oktoberfest and load packages
2. Download example data
3. Preprocess search results and spectra files
4. Annotate spectra and calibrate collision energy
5. Predict intensities and indexed retention time
6. Rescore with percolator
7. Plot and interpret the results

# 1. Installation

Before using Oktoberfest, the package and dependencies need to be installed. This step is only required once on the notebook.

## 1.1 
Install percolator.

In [ ]:
!wget https://github.com/percolator/percolator/releases/download/rel-3-06-01/percolator-v3-06-linux-amd64.deb
!dpkg -i percolator-v3-06-linux-amd64.deb

If the install fails due to missing superuser privilages, run this command instead. You will be prompted for your superuser password.

In [ ]:
import getpass
password = str(getpass.getpass()) 
!echo '{password}' | sudo -S bash -c 'dpkg -i percolator-v3-06-linux-amd64.deb'

## 1.2

Install Oktoberfest.

In [ ]:
%pip install -q oktoberfest

Import the required packages and functions by executing the code in the cell.

In [ ]:
import oktoberfest as ok
from oktoberfest import __version__ as version
import os
import pandas as pd
from pathlib import Path
import urllib.request
import requests
import shutil
from tqdm.auto import tqdm

If this works, you have installed Oktoberfest correctly. You check the version of your installed oktoberfest instance by executing the following. (This notebook was tested with Oktoberfest version 0.7.0)

In [ ]:
version

# 2. Download example data

The data used in this notebook as an example is provided as a zip archive that can be downloaded from zenodo from this record https://zenodo.org/records/10814834. In case you want to use your own data, skip ahead to Step 3.

Specify the download link as well as file name and download destination.

In [ ]:
url = "https://zenodo.org/records/10814834/files/tomato_dataset_example.zip"  # download link
download_dir = Path("./data")                                                 # choose any directory
file_name = Path("tomato_dataset_example.zip")                                # choose any filename

Download and unpack the data. (2.74G, approx. 5 minutes).

In [ ]:
if not os.path.isdir(download_dir):
  os.mkdir(download_dir)
download_file = os.path.join(download_dir, file_name)
filesize = int(requests.head(url).headers.get('content-length', -1))
with tqdm(unit="B", total=filesize, unit_scale=True, unit_divisor=1000, miniters=1, desc=url.split("/")[-1]) as t:
    urllib.request.urlretrieve(url=url, filename=download_file, reporthook=lambda blocks, block_size, _: t.update(blocks * block_size - t.n))
shutil.unpack_archive(download_file, download_dir)

# 3. Pre-Processing

## 3.1 Spectral preprocessing

### A) Converting raw files
If your spectra data is saved in form of .raw files, you need to convert them to .mzml files in order to perform rescoring. In the case that they are already in the mzml data format you can skip to step B). Since Oktoberfest uses the ThermoRawFileParser to convert the files, you need to make sure that you have installed 'mono' and give Oktoberfest the path to the executable.

Install mono if needed. Installation may differ by Linux distribution, refer to https://www.mono-project.com/download/stable/#download-lin.

In [ ]:
password = getpass.getpass()
!echo '{password}' | sudo -S bash -c 'apt install mono-complete -y'

In [ ]:
# path to the ThermoRawFileParser executable
therm = Path("/opt/compomics/ThermoRawFileParser.exe")

Get a list of all raw files located in the specified input directory.

In [ ]:
list_of_spectra = ok.pp.list_spectra(download_dir / file_name.stem,'raw')
print(list_of_spectra)

Convert the raw files to mzml files.

In [ ]:
# Define output directory where individual mzML files will be stored
mzml_directory = download_dir / "mzml/"

# Create the directory if it doesn't exist
if not os.path.exists(mzml_directory):
    os.makedirs(mzml_directory)

# Initialize an empty list to store the paths of the converted mzML files
conversion_results = []

# Loop through the list of spectra files and convert each one
for file in list_of_spectra:
    # Generate corresponding mzML file name for each raw file
    base_filename = os.path.splitext(os.path.basename(file))[0]  # Remove file extension
    output_mzml_file = os.path.join(mzml_directory, f"{base_filename}.mzML")
    
    # Call the convert_spectra function with the current file and the output file path
    ok.pp.convert_raw_to_mzml(file, output_file=output_mzml_file, thermo_exe=therm)
    
    # Append the path of the converted mzML file to the conversion_results list
    conversion_results.append(output_mzml_file)

### B) Loading Spectra
Specify the mzml path and list all mzml files in directory. In case you are using your own data, you need to change the mzml_dir variable to the path of your own mzml directory.

In [ ]:
mzml_directory = download_dir / "tomato_dataset_example" # change if needed
mzml_list = ok.pp.list_spectra(mzml_directory,'mzml')
print(mzml_list)

If you have multiple spectra files, load them all separately and save them to a list.

In [ ]:
spectra_list = []
for spectra_file in mzml_list:
    spectra = ok.pp.load_spectra(filenames=spectra_file, parser="pyteomics")
    spectra_list.append(spectra)
print(spectra_list)

## 3.2 Loading search results

Converting search results to internal Oktoberfest format. If you are using custom search results, this step isn't necessairy as they should already be provided in the internal data format. Please refer to https://oktoberfest.readthedocs.io/en/stable/internal_format.html for the specific format.

In [ ]:
# Define output directory for the converted search results
search_path = Path('./out/msms/msms.prosit')

# Create the directoriy if it doesn't exist
if not os.path.exists(search_path.parent):
    os.makedirs(search_path.parent)

# conveerting maxquant msms.txt to internal format
ok.pp.convert_search(input_path= download_dir / "tomato_dataset_example/msms.txt", output_file=search_path, search_engine='maxquant')

Loading search results to a Pandas dataframe.

In [ ]:
peptide_df = ok.pp.load_search(search_path)
peptide_df.info()

Filtering search results using given peptide length and charge requirements posed by the selected model.

In [ ]:
filtered_peptides = ok.pp.filter_peptides_for_model(peptides=peptide_df, model='prosit')

# 4. Annotate spectra and calibrate collision energy

Annotation and collision energy calibration is done seperately for each raw file so you need to loop over all spectra files in the spectra_list. 
The spectra files need to be merged on "Raw_file" and "Scan number" with the peptides taken from the search results before annotation.

In [ ]:
psm_list = []
for spectra in spectra_list:
    psms = ok.pp.merge_spectra_and_peptides(spectra=spectra, search=filtered_peptides)
    psm_list.append(psms)

Annotate the peaks for all psms in each psm_file (by efault b- and y-ions). It is important to check the mass_tolerance of the data used in order to correctly set the mass_tol and unit_mass_toll parameters. More information can be found in the docs https://oktoberfest.readthedocs.io/en/stable/config.html.

In [ ]:
library_list = []
for psms in psm_list:
    library = ok.pp.annotate_spectral_library(psms=psms, mass_tol=20, unit_mass_tol='ppm')
    library_list.append(library)

Perform ce_calibration on each annotated library with the intended model and ce range.

In [ ]:
alignment_list = []
for library in library_list:
    alignment_library = ok.pr.ce_calibration(library, ce_range=(19,50),group_by_charge=False,model_name="Prosit_2020_intensity_HCD",  
                  server_url="koina.wilhelmlab.org:443")

Determine and assign the best collision energy.

In [ ]:
for library in library_list:
    ce_alignment = alignment_library.obs.groupby(by=["COLLISION_ENERGY"])["SPECTRAL_ANGLE"].mean()
    best_ce = ce_alignment.idxmax()
    library.obs['COLLISION_ENERGY'] = best_ce

# 5. Predict intensities and indexed retention time

In [ ]:
for library in library_list:
    ok.pr.predict_rt(data = library, server_url="koina.wilhelmlab.org:443", model_name="Prosit_2019_irt", ssl=True)
    ok.pr.predict_intensities(data=library, server_url="koina.wilhelmlab.org:443", model_name="Prosit_2020_intensity_HCD", ssl=True)

# 6. Rescore with percolator

Create output dictionary for percolator results.

In [ ]:
percolator_dir = Path('./out/percolator')

if not os.path.exists(percolator_dir):
    os.makedirs(percolator_dir)

Create two feature files for each spectra files that will serve as input for percolator. The original file containing only original features from the search results as well as the rescore file containing additional features calculated by Oktoberfest.

In [ ]:
# lists to save feature files in order to merge them into one later
rescore_files = []
original_files = []

for library, spectra_file in zip(library_list,mzml_list):
    # create output paths for feature files
    original_features = percolator_dir / spectra_file.with_suffix(".original.tab").name
    original_files.append(original_features)
    rescore_features = percolator_dir / spectra_file.with_suffix(".rescore.tab").name
    rescore_files.append(rescore_features)

    # generate features for original file 
    ok.re.generate_features(library=library, search_type='original', output_file=original_features, all_features=False, regression_method='spline')
    # generate features for rescore file 
    ok.re.generate_features(library=library, search_type='rescore', output_file=rescore_features, all_features=False, regression_method='spline')

Merge all feature files into one combined original and one combined rescore file.

In [ ]:
merged_rescore = percolator_dir / "rescore.tab"
ok.re.merge_input(rescore_files, merged_rescore)

merged_original = percolator_dir / "original.tab"
ok.re.merge_input(original_files, merged_original)

Rescore with percolator.

In [ ]:
ok.re.rescore_with_percolator(merged_rescore, percolator_dir)
ok.re.rescore_with_percolator(merged_original, percolator_dir)

# Visualisations

In [ ]:
# Plots path 
plots = Path('./out/plots')

# Generate histogram of the score distribution for targets and decoys.
peptide_decoy_df = pd.read_csv(percolator_dir / 'rescore.percolator.decoy.peptides.txt', sep='\t')

peptide_target_df = pd.read_csv(percolator_dir / 'rescore.percolator.peptides.txt', sep='\t')

display(peptide_decoy_df)

ok.pl.plot_score_distribution(peptide_target_df, peptide_decoy_df, level = 'peptide', filename=plots)

In [ ]:
ok.pl.plot_all(percolator_dir)